# TensorBoardを使う
## TensorBoardの利用手順
- モデル構築時にログを取得するオペレーションを記載
- ログをマージするオペレーションを記載
- tf.summary.FileWriterを呼び出してログの出力先と対象グラフを指定
- ログを取得するタイミングでマージするオペレーションを実行
- 実行結果をFileWriterに追加
- 出力されたログをtensorboadコマンドで呼ぶ → ブラウザでアクセス

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# MNISTデータのオブジェクトを呼び出す
mnist = input_data.read_data_sets("data/", one_hot=True)

# 訓練データの取得
# ## ミニバッチを指定して、バッチ分づつデータを取得するオペレーションを作成
train_images, train_labels = mnist.train.next_batch(50)

# テスト用データの取得
test_images = mnist.test.images
test_labels = mnist.test.labels

/home/yoichi/.pyenv/versions/anaconda2-4.3.0/envs/choco3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## ログ取得オペレーションの例：Image用のログ出力

In [2]:
# 入力層
x = tf.placeholder(tf.float32, [None, 784])
# ** 入力画像をログに出力
img = tf.reshape(x, [-1,28,28,1]) # 784次元ベクトルのデータを行列に整形([batch,w,h,c])
tf.summary.image(name="input_data", tensor=img, max_outputs=10)
# ** max_outputsは、ログに出力する画像の枚数

<tf.Tensor 'input_data:0' shape=() dtype=string>

## ログ取得オペレーション：計算グラフ
- 計算グラフはname_scopeをつけて処理単位のまとまり(層など)で管理する

In [3]:
# 中間層
with tf.name_scope("hidden"):
    w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name="w1")
    b_1 = tf.Variable(tf.zeros([64]), name="b1")
    h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)
    
    # 中間層の重みの分布をログ出力
    tf.summary.histogram("w_1", w_1)
    # ** 重みを可視化する方法として、重みの分布を見る。

# 出力層
with tf.name_scope("output"):
    w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name="w2")
    b_2 = tf.Variable(tf.zeros([10]), name="b2")
    out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)

y = tf.placeholder(tf.float32, [None, 10])

# 誤差関数の定義
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y - out))
    tf.summary.scalar("loss", loss) # lossを取得するオペレーション

# 訓練
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 評価
with tf.name_scope("accuracy"):
    correct = tf.equal(tf.argmax(out,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    tf.summary.scalar("accuracy", accuracy) # accuracyを取得するオペレーション


## ログのマージ
- ログ取得用のオペレーションを複数書いたので、ログをマージして1度に全てのログを取得したい
- マージしないと、それぞれのログを個別に取得する必要がある

In [4]:
summary_op = tf.summary.merge_all()

## ログの書き込み

In [5]:
# 初期化
init = tf.global_variables_initializer()

# 実行
with tf.Session() as sess:
    # ** 計算グラフをGraphsに出力
    summary_writer = tf.summary.FileWriter("logs", sess.graph)
    
    sess.run(init)
    
    #テストデータをロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels
    
    for i in range(1000):
        step = i + 1
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images, y:train_labels})
        
        if step % 10 == 0:
            # ログを取得する処理を実行
            summary_str = sess.run(summary_op, feed_dict={x:test_images, y:test_labels})
            # ログ情報のプロトコルバッファを書き込む
            summary_writer.add_summary(summary_str, step)
            
            #acc_val = sess.run(accuracy, feed_dict={x:test_images, y:test_labels})
            #print('Step %d: accuracy = %.2f'%(step, acc_val))
            

## ログの確認
- 以下のコマンド
```
$ tensorboard --logdir ./logs/
```
    - 引数はログディレクトリ
    - defaultではポート6006でブラウザから確認できる